# Assignment 12.1 - Graph Neural Networks

Please submit your solution of this notebook in the Whiteboard at the corresponding Assignment entry as .ipynb-file and as .pdf.

#### Please state both names of your group members here:
Farah Ahmed Atef Abdelhameed Hafez

## Task 12.1.1: GNNs

* Implement a simple Graph Convolutional Network. **(RESULT)**
* Train and evaluate it on the Cora dataset provided below. **(RESULT)**

Hint: you might need to install torch_geometric first

`pip install torch_geometric`

In [1]:
pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

# Load Cora dataset (citation network: 2708 papers, 7 classes)
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
data.x = F.normalize(data.x, p=2, dim=1)

Processing...
Done!


#Build Model

In [11]:
class GCN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(GCN, self).__init__()
    self.conv1 = GCNConv(input_dim, hidden_dim)
    self.conv2 = GCNConv(hidden_dim, output_dim)
  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x,  p=0.6, training=self.training)
    x = self.conv2(x, edge_index)
    return x


#Train Logic

In [22]:
def train(model, data, optimizer, criterion, epochs):
  model.train()
  for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print("Epoch:", epoch," Loss:", loss.item())



#Test Logic

In [19]:
def test(model, data):
  model.eval()
  out = model(data.x, data.edge_index)
  _, pred = out.max(dim=1)
  correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
  acc = correct / data.test_mask.sum().item()
  print(f'Test Accuracy: {acc:.4f}')


# Train and Test

In [23]:
model=GCN(dataset.num_features, 64, dataset.num_classes)
optimizer=torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion=torch.nn.CrossEntropyLoss()
train(model, data, optimizer, criterion, 100)
test(model, data)


Epoch: 0  Loss: 1.9450362920761108
Epoch: 1  Loss: 1.891877293586731
Epoch: 2  Loss: 1.8204114437103271
Epoch: 3  Loss: 1.7387816905975342
Epoch: 4  Loss: 1.6449443101882935
Epoch: 5  Loss: 1.5460402965545654
Epoch: 6  Loss: 1.4347070455551147
Epoch: 7  Loss: 1.3445169925689697
Epoch: 8  Loss: 1.2171192169189453
Epoch: 9  Loss: 1.0922983884811401
Epoch: 10  Loss: 1.0277953147888184
Epoch: 11  Loss: 0.9050562381744385
Epoch: 12  Loss: 0.8025797009468079
Epoch: 13  Loss: 0.7205952405929565
Epoch: 14  Loss: 0.6401546001434326
Epoch: 15  Loss: 0.5530844330787659
Epoch: 16  Loss: 0.4962339699268341
Epoch: 17  Loss: 0.4607253968715668
Epoch: 18  Loss: 0.3812163770198822
Epoch: 19  Loss: 0.335071861743927
Epoch: 20  Loss: 0.3078550398349762
Epoch: 21  Loss: 0.25346577167510986
Epoch: 22  Loss: 0.25134265422821045
Epoch: 23  Loss: 0.20467105507850647
Epoch: 24  Loss: 0.20017313957214355
Epoch: 25  Loss: 0.1772502213716507
Epoch: 26  Loss: 0.15976834297180176
Epoch: 27  Loss: 0.1500134319067001

## Congratz, you made it! :)